#Previsione Livelli Ricoveri Terapia Intensiva, Nuovi Positivi e Deceduti per COVID

Nel seguente notebook sono stati utilizzati tre diversi modelli, LSTM, XGBoost e ARIMA con i dati resi disponibili dal Ministero della Salute per effettuare un forecast con orizzonte 1, 2, 7 e 14 giorni dei valori considerati.

Sono stati usati i dati regionali resi disponibili via file .csv al [link](https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv) e filtrati per regione di interesse, in questo caso Emilia-Romagna.
Ogni riga è un aggiornamento giornaliero dei seguenti valori: 
data, stato, codice_regione, denominazione_regione, lat, long, ricoverati_con_sintomi, terapia_intensiva,totale_ospedalizzati, isolamento_domiciliare, totale_positivi, variazione_totale_positivi, nuovi_positivi, dimessi_guariti, deceduti, casi_da_sospetto_diagnostico, casi_da_screening, totale_casi, tamponi, casi_testati, note, ingressi_terapia_intensiva, note_test, note_casi, totale_positivi_test_molecolare, totale_positivi_test_antigenico_rapido, tamponi_test_molecolare, tamponi_test_antigenico_rapido, codice_nuts_1, codice_nuts_2. 

Di questi sono state poi utilizzate per l'analisi: 
data, ricoverati_con_sintomi, terapia_intensiva, totale_ospedalizzati, variazione_totale_positivi, nuovi_positivi, deceduti, tamponi, ingressi_terapia_intensiva.

I valori di deceduti e tamponi sono stati differenziati per avere l'incremento giornaliero invece del dato cumulativo che era presente.

L'analisi svolta è su singola variabile.

I dati sono stati divisi con percentuale di 80 e 20 rispettivamente per training e testing.

Il primo 80% di dati in fase di tuning è a sua volta divisa in 80% di train e 20% di validation, in modo che il tuning non venga effettuato su dati di test.

Dopo la suddivisione è stato effettuato lo scaling attravero un MinMaxScaler che comprime i valori tra -1 ed 1 (la trasformazione viene poi invertita dopo la previsione).

Per i modelli XGBoost e LSTM sono stati preparati i dati creando per ogni giorno un array di lag temporali e di valori futuri della variabile presa in considerazione:
(t-n,...,t-1,t) -> (t+1,...,t+p) con n = giorni di lag e p = orizzonte di previsione.
Questi valori sono stati utilizzati per il training dei due modelli utilizzando le variabili lag come ingresso del modello e i valori futuri come target.

Con ARIMA per ogni giorno la previsione degli n giorni successivi è stata calcolata fornendo al modello i dati presenti fino a quel momento, i quali vengono usati per calcolare i coefficienti usati per effettuare la previsione. Il modello scelto per ARIMA è con (p,q,d) = (14,1,1). Sono stati esclusi i primi 500 giorni di serie temporale.

L'errore considerato è il Mean Absolute Error sull'orizzonte di previsione. Viene poi considerata la media dei MAE calcolati su tutte le previsioni.

Gli iper-parametri sono stati scelti attraverso una grid search per ogni variabile e per ogni orizzonte temporale selezionando il modello che da una media dei MAE inferiore.

Il grafico mostra lo slot temporale di previsione con MAE inferiore.


In [43]:
import os

if 'google.colab' in str(get_ipython()):
    files = ['util.py']

    for file in files:
        os.system('rm ./' + file)
        os.system(
            'wget -nv https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/' + file)

import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from numpy.random import seed
import plotly.graph_objects as go

# fix for 'package not found' when installing in Anaconda environment
if 'google.colab' not in str(get_ipython()):
    import pip
    pip.main(['install', 'xgboost'])

from xgboost import XGBRegressor
from util import select_relevant_rows, select_attributes, read_movement_data, download_updated_mobility_data, download_updated_mobility_data, save_config, load_config


In [44]:
use_existing_config = True
column_to_predict = 'terapia_intensiva'
columns = ['nuovi_positivi', 'terapia_intensiva', 'deceduti']
split_percent = 0.80
region_focus = 'Emilia-Romagna'
attribute_focus = 'denominazione_regione'
n_futures = [1, 2, 7, 14]

In [45]:
local_region_path = r'../COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv'
remote_region_path = r'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

regions_frame = pd.read_csv(remote_region_path)

region_focus_data = select_relevant_rows(
    regions_frame,
    attribute_focus,
    region_focus
)

frame_interesting_columns = select_attributes(region_focus_data, [
    'data',
    'ricoverati_con_sintomi',
    'terapia_intensiva',
    'totale_ospedalizzati',
    'variazione_totale_positivi',
    'nuovi_positivi',
    'deceduti',
    'tamponi',
    'ingressi_terapia_intensiva'
])

frame_interesting_columns = pd.DataFrame(frame_interesting_columns)
frame_interesting_columns['data'] = pd.to_datetime(
    frame_interesting_columns['data'])
frame_interesting_columns['data'] = frame_interesting_columns['data'].dt.strftime(
    r'%Y-%m-%d')
frame_interesting_columns['data'] = pd.to_datetime(frame_interesting_columns['data'])
frame_interesting_columns = frame_interesting_columns[frame_interesting_columns['data'] < pd.to_datetime('2022-01-31')]
frame_interesting_columns = frame_interesting_columns.fillna(0)

frame_interesting_columns.rename(columns={'data': 'date'}, inplace=True)
frame_interesting_columns.set_index('date', inplace=True)

# revert cumulative data
frame_interesting_columns['deceduti'] = frame_interesting_columns['deceduti'].diff(
)
frame_interesting_columns['tamponi'] = frame_interesting_columns['tamponi'].diff(
)
frame_interesting_columns.dropna(inplace=True)

frame_interesting_columns.tail

# numpy seed
seed(1)

frame_interesting_columns = frame_interesting_columns[500:]

In [46]:
def split_series(series, n_past, n_future, arima=False):
    X, y, X_indexes, y_indexes = list(), list(), list(), list()
    index = np.array(series.index).reshape(series.values.shape[0], 1)
    series = series.values

    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        start = 0 if arima == True else window_start

        past, future = series[start:past_end,
                              :], series[past_end:future_end, :]
        past_index, future_index = index[start:past_end,
                                         :], index[past_end:future_end, :]
        X.append(past)
        y.append(future)
        X_indexes.append(past_index)
        y_indexes.append(future_index)

    return np.array(X), np.array(y), np.array(X_indexes), np.array(y_indexes)


def plot_best_pred(
    sorted_results, column_to_predict):
        pred = sorted_results[0][1][-1]['pred']
        test = sorted_results[0][1][-1]['y_test']

        prediction_trace = go.Scatter(
            x=pred.index, y=pred, mode='lines', name='Prediction')
        truth_trace = go.Scatter(
            x=test.index, y=test, mode='lines', name='Ground Truth')
        layout = go.Layout(
            title=column_to_predict, xaxis={'title': 'Date'},
            yaxis={'title': column_to_predict})
        fig = go.Figure(
            data=[prediction_trace, truth_trace], layout=layout)
        fig.show()

#ARIMA Multi Output

In [47]:
def execute_arima(dataframe, order, column_to_predict, split_percent, n_future=7):
    df = dataframe[column_to_predict].copy()
    split = int(split_percent*len(df))

    n_past = 14
    n_features = 1

    train, test = pd.DataFrame(df[:split]), pd.DataFrame(df[split:])

    scalers = {}

    for i in train.columns:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        s_s = scaler.fit_transform(train[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_' + i] = scaler
        train[i] = s_s

    for i in test.columns:
        scaler = scalers['scaler_'+i]
        s_s = scaler.transform(test[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_'+i] = scaler
        test[i] = s_s

    X_test, y_test, X_test_indexes, y_test_indexes = split_series(
        test, n_past, n_future, arima=True)
    
    df_results = []

    history = [x for x in train.values]

    for i in range(len(X_test)):
        current_history = np.array(history).reshape(-1)
        current_history = np.append(current_history, X_test[i].reshape(-1))

        model = ARIMA(current_history, order=order)
        model_fitted = model.fit()
        prediction = model_fitted.forecast(n_future)[0]

        prediction = scaler.inverse_transform(prediction.reshape(-1,1))
        y_test[i] = scaler.inverse_transform(y_test[i])

        current = pd.DataFrame(
            {'y_test':y_test[i].reshape(-1),
            'pred':prediction.reshape(-1),
            'dates':y_test_indexes[i].reshape(-1)})

        current.set_index('dates', inplace=True)
        
        df_results.append(current)

    results = {}

    for el in df_results:
        mae = mean_absolute_error(el['y_test'], el['pred'])
        mape = mean_absolute_percentage_error(el['y_test'], el['pred'])
        results[el.index[0]] = (mae, mape, el)

    sorted_results = sorted(results.items(), key=lambda x: x[1][0])

    return sorted_results


def define_arima_configs():
    p_values = [10]
    d_values = [1]
    q_values = [1]
    return p_values, d_values, q_values


def evaluate_models(dataframe, column_to_predict, split_percent, n_future=7):
    p_values, d_values, q_values = define_arima_configs()
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p, d, q)
                sorted_results = execute_arima(
                    dataframe, order, column_to_predict, split_percent, n_future)
                avg_error = np.mean(
                    np.array(list(map(lambda x:x[1][0], sorted_results))))
                if avg_error < best_score:
                    best_score, best_cfg = avg_error, order
    return best_cfg

use_existing_congfig = True
summaries = []

for n_future in n_futures:
    for column_to_predict in columns:
        config_path = region_focus + '_' + 'arima_config' + '_' + column_to_predict
        config_path = config_path + '_' + str(n_future)
        if use_existing_config:
            # if not os.path.isfile(config_path):
            #     os.system('wget -nv https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/configs/' + config_path)
            # config = load_config(config_path)
            config = (14,1,1)
            try:
                sorted_results = execute_arima(
                    frame_interesting_columns, config, column_to_predict, split_percent, n_future=n_future)
            except:
                sorted_results = None
            # os.system('rm ' + config_path)
        else:
            config = evaluate_models(
                frame_interesting_columns, column_to_predict, split_percent)

            save_config(config_path, config)

            sorted_results = execute_arima(
                frame_interesting_columns, config, column_to_predict, split_percent, n_future=n_future)

        summary = ''
        if sorted_results is None:
            summary = '|' + column_to_predict + '| seq len ' + str(n_future) + ' no AR found'
        else:
            avg_mae = np.mean(np.array(list(map(lambda x:x[1][0], sorted_results))))
            avg_mape = np.mean(np.array(list(map(lambda x:x[1][1], sorted_results))))

            summary = '|' + column_to_predict + '| seq len ' + str(n_future) + '| mae: ' + str(avg_mae) + '| mape: ' + str(avg_mape) + '| best config: ' + str(config)

        print(summary)
        summaries.append(summary)

        if sorted_results is not None:
            if n_future > 2:
                plot_best_pred(sorted_results, column_to_predict)
            else:
                print(('Pred: ', sorted_results[0][1][-1]['pred'].values))
                print(('Test: ', sorted_results[0][1][-1]['y_test'].values))

print(summaries)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


|nuovi_positivi| seq len 1 no AR found
|terapia_intensiva| seq len 1| mae: 3.156123934595782| mape: 0.021423700430135622| best config: (14, 1, 1)
('Pred: ', array([150.21555792]))
('Test: ', array([150.]))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:695: RuntimeWarning: divide by zero encountered in true_divide
  invmacoefs = -np.log((1-macoefs)/(1+macoefs))
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:668: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:669: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()


|deceduti| seq len 1 no AR found
|nuovi_positivi| seq len 2 no AR found


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



|terapia_intensiva| seq len 14| mae: 9.59858638836324| mape: 0.0648045869196027| best config: (14, 1, 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



|deceduti| seq len 14| mae: 10.087469176395267| mape: 0.3115125883956501| best config: (14, 1, 1)


['|nuovi_positivi| seq len 1 no AR found', '|terapia_intensiva| seq len 1| mae: 3.156123934595782| mape: 0.021423700430135622| best config: (14, 1, 1)', '|deceduti| seq len 1 no AR found', '|nuovi_positivi| seq len 2 no AR found', '|terapia_intensiva| seq len 2| mae: 3.4858362703298535| mape: 0.023725512601582385| best config: (14, 1, 1)', '|deceduti| seq len 2 no AR found', '|nuovi_positivi| seq len 7 no AR found', '|terapia_intensiva| seq len 7| mae: 5.314013706647928| mape: 0.03597405425436039| best config: (14, 1, 1)', '|deceduti| seq len 7 no AR found', '|nuovi_positivi| seq len 14 no AR found', '|terapia_intensiva| seq len 14| mae: 9.59858638836324| mape: 0.0648045869196027| best config: (14, 1, 1)', '|deceduti| seq len 14| mae: 10.087469176395267| mape: 0.3115125883956501| best config: (14, 1, 1)']
